To run it just push the play button.

In [3]:
# By Werki 13.01.2024
import json
import os.path

info_tabelle = {"+X_+Y": "X_Y-_X-_Y",  # Eine Tabelle die der code benötigt, um Richtungen herauszufinden
                "+X_-Y": "X_Y_X-_Y-",
                "-X_+Y": "X-_Y-_X_Y",
                "-X_-Y": "X-_Y_X_Y-",
                "+Y_+X": "Y_X-_Y-_X",
                "+Y_-X": "Y_X_Y-_X-",
                "-Y_+X": "Y-_X-_Y_X",
                "-Y_-X": "Y-_X_Y_X-"}

# TODO: Werkzeugradius korrektur


def get_user_input():
    """Erfragt die eingaben des Benutzers"""
    input_temp = input("Test (j/n)?: ").lower()
    if input_temp == "j" or input_temp == "y" or input_temp == "":
        with open("test.json", "r") as f:
            output_user = json.load(f)

    else:
        vorschub_ges = int(input("Vorschub (mm/min): "))
        zustellung = float(input("Zustellung (mm): "))
        schnitt_tiefe = int(input("Tiefe des Schnitts (mm): "))
        spindel_ges = int(input("Spindel geschwindigkeit (rpm): "))
        len_des_schnitts = int(input("Länge des Schnitts (mm): "))
        x_position_schnitt_anfang = int(input("X Position Schnitt Anfang (mm): "))
        y_position_schnitt_anfang = int(input("Y Position Schnitt Anfang (mm): "))
        z_position_schnitt_anfang = int(input("z Position Schnitt Anfang (mm): "))
        richtung_des_schnitts = input("Richtung des Schnitts (+X, -X, +Y, -Y): ").upper()
        lage_des_materials = input("Lage des Materials (+X, -Y, +Y, -Y): ").upper()
        datei_name = input("Dateiname(str): ").replace(" ", "")
        sicherheitsabstand = int(input("sicherheitsabstand (größer als fräser radius) (mm): "))
        output_user: dict = {"vorschub_ges": vorschub_ges, "zustellung": zustellung, "schnitt_tiefe": schnitt_tiefe, "spindel_ges": spindel_ges, "len_des_schnitts": len_des_schnitts,
                             "x_position_schnitt_anfang": x_position_schnitt_anfang, "y_position_schnitt_anfang": y_position_schnitt_anfang, "z_position_schnitt_anfang": z_position_schnitt_anfang,
                             "richtung_des_schnitts": richtung_des_schnitts, "lage_des_materials": lage_des_materials, "datei_name": datei_name, "sicherheitsabstand": sicherheitsabstand} # NOQA
        print("\n\n")
    return output_user


def write_file(name, cont):
    """Schreibt den fertigen Gcode in eine datei."""

    if os.path.isfile(f"./{name}.nc"):
        print("Datei existiert! nicht geschrieben")
        if input("Trotzdem schreiben (y/n)?: ").lower() == "n":
            return
        else:
            with open(f"{name}.nc", "w") as f: # NOQA
                pass
            with open(f"{name}.nc", "a") as f:
                for i in cont:
                    f.write(i)
                    f.write("\n")
            print("erfolgreich geschrieben")
    else:
        with open(f"{name}.nc", "a") as f:
            for i in cont:
                f.write(i)
                f.write("\n")
        print("erfolgreich geschrieben")


def cut_para(input_user, info_liste):
    """Benutzt die eingaben des Benutzers mithilfe einer liste, um herauszufinden,
    in welcher seite weggestellt werden kann"""

    list_search = f"{input_user['richtung_des_schnitts']}_{input_user['lage_des_materials']}"
    result = info_liste[list_search].split("_")
    return result


def cycle_define(input_user, info_liste):
    """Schreibt den Gcode in eine liste und gib dies zurück"""

    output: list[str] = []
    len_des_schnitts = input_user["len_des_schnitts"] + input_user["sicherheitsabstand"] + input_user["sicherheitsabstand"] + input_user["zustellung"]

    for key, value in input_user.items():
        output.append(f"(Einstellung: {key} = {value})")
    output.append("\n\nG90 (Absolute Positionierung)\n")
    output.append(f"G0 X{input_user['x_position_schnitt_anfang'] - input_user['sicherheitsabstand'] - input_user['zustellung']} Y{input_user['y_position_schnitt_anfang'] - input_user['sicherheitsabstand'] - input_user['zustellung']} Z{input_user['z_position_schnitt_anfang']}") # NOQA

    output.append(f"\nG91 (Relative Positionierung)\n")
    output.append(f"M03 S{input_user['spindel_ges']}")

    # Eigentliche Logik

    para_list = cut_para(input_user, info_liste)
    passes, rest = divmod(input_user["schnitt_tiefe"], input_user["zustellung"])
    while passes != 0:
        output.append(f"G1 {para_list[3]}{input_user['sicherheitsabstand'] + input_user['zustellung']} F{input_user['vorschub_ges']}")
        output.append(f"G1 {para_list[0]}{len_des_schnitts} F{input_user['vorschub_ges']}")
        output.append(f"G0 {para_list[1]}{input_user['sicherheitsabstand']}")
        output.append(f"G0 {para_list[2]}{len_des_schnitts}\n")
        passes -= 1

    output.append(f"G1 {para_list[3]}{input_user['sicherheitsabstand'] + rest} F{input_user['vorschub_ges']}")
    output.append(f"G1 {para_list[0]}{len_des_schnitts} F{input_user['vorschub_ges']}")
    output.append(f"G0 {para_list[1]}{input_user['sicherheitsabstand']}")
    output.append(f"G0 {para_list[2]}{len_des_schnitts}\n")
    output.append(f"\nM05")
    output.append(f"\nM30\n")

    print("\n")
    print("[Output]:")
    for i in output:
        print(f"{i}")
    return output


if __name__ == '__main__':
    temp1 = get_user_input()
    temp2 = cycle_define(temp1, info_tabelle)
    write_file(temp1["datei_name"], temp2)


Test (j/n)?: y


[Output]:
(Einstellung: vorschub_ges = 10)
(Einstellung: zustellung = 0.5)
(Einstellung: schnitt_tiefe = 5)
(Einstellung: spindel_ges = 500)
(Einstellung: len_des_schnitts = 10)
(Einstellung: x_position_schnitt_anfang = 10)
(Einstellung: y_position_schnitt_anfang = 10)
(Einstellung: z_position_schnitt_anfang = 5)
(Einstellung: richtung_des_schnitts = +X)
(Einstellung: lage_des_materials = +Y)
(Einstellung: datei_name = for_test_)
(Einstellung: sicherheitsabstand = 1)


G90 (Absolute Positionierung)

G0 X8.5 Y8.5 Z5

G91 (Relative Positionierung)

M03 S500
G1 Y1.5 F10
G1 X12.5 F10
G0 Y-1
G0 X-12.5

G1 Y1.5 F10
G1 X12.5 F10
G0 Y-1
G0 X-12.5

G1 Y1.5 F10
G1 X12.5 F10
G0 Y-1
G0 X-12.5

G1 Y1.5 F10
G1 X12.5 F10
G0 Y-1
G0 X-12.5

G1 Y1.5 F10
G1 X12.5 F10
G0 Y-1
G0 X-12.5

G1 Y1.5 F10
G1 X12.5 F10
G0 Y-1
G0 X-12.5

G1 Y1.5 F10
G1 X12.5 F10
G0 Y-1
G0 X-12.5

G1 Y1.5 F10
G1 X12.5 F10
G0 Y-1
G0 X-12.5

G1 Y1.5 F10
G1 X12.5 F10
G0 Y-1
G0 X-12.5

G1 Y1.5 F10
G1 X12.5 F10
G0 Y-1
G0